Para poder trabajar con DataFrames necesitamos importar numpy y pandas.

El objetivo de este notebook es combinar dos tablas y luego dividir la tabla final en archivos de 30000 lineas como máximo.

In [1]:
import numpy as np
import pandas as pd

Leemos el primer csv con solamente con las columnas que necesitamos y utilizando dtype='str' para que todas las columnas
tengan formato string.

En una versión anterior creaba una columna y a través de lambda cambiaba el formato.

In [2]:
cruzar = pd.read_csv('file2.csv', usecols=['COLUMN','ID'], dtype='str')

cruzar['COLUMN_2'] = cruzar['COLUMN'].map(lambda x: str(x))

Uno de los archivos que voy a combinar no tiene header, entonces necesito cogerlo de un archivo similiar.
Y posteriormente lo paso a una lista.

In [4]:
headertabla = pd.read_csv('header2.csv',delimiter =';',encoding='latin-1')

C:\Users\corrales\AppData\Local\Continuum\anacond3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
header = headertabla.columns.tolist()

Leemos el archivo que está en formato TXT, al principio no se podía leer pero con encoding='latin-1' se puede.
Y le damos nombre a las columnas anteriormente extraidas.

In [6]:
Empresa = pd.read_csv('file.txt', delimiter=';', encoding='latin-1', dtype='str')

In [7]:
Empresa.columns = header

Hacemos un proceso parecido debido a que en otro archivo tenemos tenemos solamente las columnas que queremos del anterior
archivo, entonces creamos una nueva variable solamente con las columnas que necesitamos.

In [8]:
header2 = pd.read_csv('header.csv', delimiter=';', encoding='latin-1')

In [9]:
headerfinal = header2.columns.tolist()

In [10]:
Empresafinal = Empresa[headerfinal]

Quitamos los NaN de la columna key porque no los necesitamos.

In [11]:
Empresasconmatr = Empresafinal.dropna(subset=['key'])

Combinamos las dos tablas a través de las columnas que tienen valores idénticos.

In [12]:
VehiculosconID = Empresasconmatr.merge(cruzar, how ='left', left_on = ['id_2'], right_on = ['ID'])

Quitamos NaN de 'COLUMN' y duplicados de 'key'

In [13]:
VehiculosconIDfinal = VehiculosconID.dropna(subset=['COLUMN']).drop_duplicates(subset=['key'])

Creamos una nueva columna con un nombre.

In [14]:
nombre_columna = 'Name'

In [15]:
VehiculosconIDfinal['New_column'] = nombre_columna

Creamos un archivo que lo llamaremos temporal porque posteriormente lo iremos sobrescribiendo.

In [18]:
temporal = VehiculosconIDfinal.to_csv('temporal.csv', encoding='latin-1', float_format='string')

In [19]:
reader = pd.read_csv('temporal.csv', delimiter=',', encoding='latin-1', chunksize=29998, dtype='str')

Leemos el archivo temporal con chunks de 29998 para posteriormente hacer un loop.

Creamos tantos archivos como chunks de 29998 tengamos con el nombre de la columna que hemos dado antes
y con un contado porque sino se irán pisando los archivos.

In [20]:
count = 0
for chunk in reader:
    count += 1
    count2 = str(count)
    name_file ='%s%s.csv' %(nombre_columna,count2)
    chunk.to_csv(name_file, encoding='latin-1', float_format='string')